In [1]:
import requests
import pandas as pd
import my_api_key as key
import csv

In [3]:
api_key = key.api_key

In [4]:
df = pd.read_csv("gps_data.csv")
df.head()


,Ville,Latitude,Longitude
0,Mont Saint Michel,48.635954,-1.511460
1,Saint Malo,48.649518,-2.026041
2,Bayeux,49.276462,-0.702474
3,Le Havre,49.493898,0.107973
4,Rouen,49.440459,1.093966


In [5]:
import csv

# Fonction pour récupérer la température et le champ "main" d'une ville
def get_temperature_and_main(latitude, longitude):
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    if "current" in data:
        temperature_celsius = data["current"]["temp"] - 273.15  # Convertir de Kelvin en Celsius
        weather_main = data["current"]["weather"][0]["main"]
        return temperature_celsius, weather_main
    else:
        return None, None

Les résultats ont été enregistrés dans 'temperature.csv'


In [ ]:
csv_filename = "meteo_data.csv"

# Créer un fichier CSV
with open('meteo_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['Ville', 'Température (°C)', 'Météo principale']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Parcourir les lignes du DataFrame
    for index, row in df.iterrows():
        ville = row['Ville']
        latitude = row['Latitude']
        longitude = row['Longitude']
        temperature, weather_main = get_temperature_and_main(latitude, longitude)
        if temperature is not None:
            temperature_rounded = round(temperature)  # Arrondir la température à l'entier le plus proche
            writer.writerow({'Ville': ville, 'Température (°C)': temperature_rounded, 'Météo principale': weather_main})
            print(f"Ville: {ville}, Température : {temperature_rounded}°C, Météo principale: {weather_main}")
        else:
            writer.writerow({'Ville': ville, 'Température (°C)': 'N/A', 'Météo principale': 'N/A'})

print("Les résultats ont été enregistrés dans 'meteo_data.csv'")

In [7]:
df = pd.read_csv("meteo_data.csv")
df.head()

,Ville,Température (°C),Météo
0,Mont Saint Michel,16,Clouds
1,Saint Malo,16,Rain
2,Bayeux,17,Clouds
3,Le Havre,16,Clouds
4,Rouen,18,Clouds
